### 加载数据集

In [1]:
from utils import load_corpus, stopwords

TRAIN_PATH = "./data/weibo2018/train.txt"
TEST_PATH = "./data/weibo2018/test.txt"

In [2]:
# 分别加载训练集和测试集
train_data = load_corpus(TRAIN_PATH)
test_data = load_corpus(TEST_PATH)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ghost\AppData\Local\Temp\jieba.cache
Loading model cost 0.550 seconds.
Prefix dict has been built successfully.


In [3]:
import pandas as pd

df_train = pd.DataFrame(train_data, columns=["text", "label"])
df_test = pd.DataFrame(test_data, columns=["text", "label"])
df_train.head()

,text,label
0,书中 自有 黄金屋 书中 自有 颜如玉 沿着 岁月 的 长河 跋涉 或是 风光旖旎 或是 姹...,1
1,这是 英超 被 黑 的 最惨 的 一次 二哈 二哈 十几年来 中国 只有 孙继海 董方卓 郑...,0
2,中国 远洋 海运 集团 副总经理 俞曾 港 月 日 在 上 表示 中央 企业 走 出去 是 ...,1
3,看 流星花园 其实 也 还好 啦 现在 的 观念 以及 时尚 眼光 都 不一样 了 或许 十...,1
4,汉武帝 的 罪己 诏 的 真实性 尽管 存在 着 争议 然而 轮台 罪己 诏 作为 中国 历...,1


### 训练词向量

In [4]:
# word2vec要求的输入格式: list(word)
wv_input = df_train['text'].map(lambda s: s.split(" "))   # [for w in s.split(" ") if w not in stopwords]
wv_input.head()                         

0    [书中, 自有, 黄金屋, 书中, 自有, 颜如玉, 沿着, 岁月, 的, 长河, 跋涉, ...
1    [这是, 英超, 被, 黑, 的, 最惨, 的, 一次, 二哈, 二哈, 十几年来, 中国,...
2    [中国, 远洋, 海运, 集团, 副总经理, 俞曾, 港, 月, 日, 在, 上, 表示, ...
3    [看, 流星花园, 其实, 也, 还好, 啦, 现在, 的, 观念, 以及, 时尚, 眼光,...
4    [汉武帝, 的, 罪己, 诏, 的, 真实性, 尽管, 存在, 着, 争议, 然而, 轮台,...
Name: text, dtype: object

In [5]:
from gensim import models

# Word2Vec
word2vec = models.Word2Vec(wv_input, 
                           vector_size=64,   # 词向量维度
                           min_count=1,      # 最小词频, 因为数据量较小, 这里卡1
                           epochs=1000)      # 迭代轮次

查找近义词, 直观感受训练得到的word2vec效果

In [6]:
word2vec.wv.most_similar("你")

[('我', 0.9429759383201599),
 ('自己', 0.8931552171707153),
 ('他', 0.8775283098220825),
 ('的', 0.8603615164756775),
 ('她', 0.8426206111907959),
 ('人', 0.8322935104370117),
 ('都', 0.8209617137908936),
 ('了', 0.7984644770622253),
 ('就', 0.7978600263595581),
 ('也', 0.7877569794654846)]

In [7]:
word2vec.wv.most_similar("哈哈")

[('哈哈哈', 0.662361741065979),
 ('啦', 0.547860860824585),
 ('可爱', 0.5176939964294434),
 ('本柔', 0.4890766441822052),
 ('又', 0.47497376799583435),
 ('了', 0.4720135033130646),
 ('回家', 0.46364885568618774),
 ('摊手', 0.4579325020313263),
 ('呀', 0.4526076018810272),
 ('呵呵', 0.4457685053348541)]

In [8]:
word2vec.wv.most_similar("伤心")

[('难过', 0.7174865007400513),
 ('哭', 0.6679071187973022),
 ('想', 0.6562684774398804),
 ('也', 0.6426547169685364),
 ('痛苦', 0.6406323313713074),
 ('真的', 0.6282573938369751),
 ('的', 0.615120530128479),
 ('我', 0.6093114018440247),
 ('开心', 0.6055984497070312),
 ('都', 0.6049546599388123)]

### 神经网络

In [9]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence,pad_packed_sequence
from torch.utils.data import Dataset, DataLoader

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [10]:
# 超参数
learning_rate = 5e-4
input_size = 768
num_epoches = 5
batch_size = 100
embed_size = 64
hidden_size = 64
num_layers = 2

In [11]:
# 数据集
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = []
        self.label = df["label"].tolist()
        for s in df["text"].tolist():
            vectors = []
            for w in s.split(" "):
                if w in word2vec.wv.key_to_index:
                    vectors.append(word2vec.wv[w])   # 将每个词替换为对应的词向量
            vectors = torch.Tensor(vectors)
            self.data.append(vectors)
    
    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]
        return data, label

    def __len__(self):
        return len(self.label)

def collate_fn(data):
    """
    :param data: 第0维：data，第1维：label
    :return: 序列化的data、记录实际长度的序列、以及label列表
    """
    data.sort(key=lambda x: len(x[0]), reverse=True) # pack_padded_sequence要求要按照序列的长度倒序排列
    data_length = [len(sq[0]) for sq in data]
    x = [i[0] for i in data]
    y = [i[1] for i in data]
    data = pad_sequence(x, batch_first=True, padding_value=0)   # 用RNN处理变长序列的必要操作
    return data, torch.tensor(y, dtype=torch.float32), data_length


# 训练集
train_data = MyDataset(df_train)
train_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

# 测试集
test_data = MyDataset(df_test)
test_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

C:\Users\ghost\AppData\Local\Temp\ipykernel_16544\2810783818.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  vectors = torch.Tensor(vectors)


In [17]:
# 网络结构
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, 1)  # 双向, 输出维度要*2
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)  # 双向, 第一个维度要*2
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        
        packed_input = torch.nn.utils.rnn.pack_padded_sequence(input=x, lengths=lengths, batch_first=True)
        packed_out, (h_n, h_c) = self.lstm(packed_input, (h0, c0))

        lstm_out = torch.cat([h_n[-2], h_n[-1]], 1)  # 双向, 所以要将最后两维拼接, 得到的就是最后一个time step的输出
        out = self.fc(lstm_out)
        out = self.sigmoid(out)
        return out

lstm = LSTM(embed_size, hidden_size, num_layers).to(device)

In [21]:
from sklearn import metrics

def test():
    y_pred_list, y_true_list = [], []

    with torch.no_grad():
        for x, labels, lengths in test_loader:
            x = x.to(device)
            outputs = lstm(x, lengths)         # 前向传播
            outputs = outputs.view(-1)         # 将输出展平
            y_pred_list.append(outputs)
            y_true_list.append(labels)

    y_prob = torch.cat(y_pred_list)
    y_true = torch.cat(y_true_list)
    
    # 将预测值二值化
    y_pred = y_prob.clone()
    y_pred[y_pred > 0.5] = 1
    y_pred[y_pred <= 0.5] = 0

    # 在使用 scikit-learn 方法前，将张量搬回 CPU 并转换为 numpy 数组
    y_prob_np = y_prob.cpu().numpy()
    y_true_np = y_true.cpu().numpy()
    y_pred_np = y_pred.cpu().numpy()

    print(metrics.classification_report(y_true_np, y_pred_np))
    print("准确率:", metrics.accuracy_score(y_true_np, y_pred_np))
    print("AUC:", metrics.roc_auc_score(y_true_np, y_prob_np))


In [22]:
# 定义损失函数和优化器
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

In [23]:
# 迭代训练
for epoch in range(num_epoches):
    total_loss = 0
    for i, (x, labels, lengths) in enumerate(train_loader):
        x = x.to(device)
        labels = labels.to(device)
        outputs = lstm(x, lengths)          # 前向传播
        logits = outputs.view(-1)           # 将输出展平
        loss = criterion(logits, labels)    # loss计算
        total_loss += loss
        optimizer.zero_grad()               # 梯度清零
        loss.backward(retain_graph=True)    # 反向传播，计算梯度
        optimizer.step()                    # 梯度更新
        if (i+1) % 10 == 0:
            print("epoch:{}, step:{}, loss:{}".format(epoch+1, i+1, total_loss/10))
            total_loss = 0
    
    # test
    test()
    
    # save model
    model_path = "./model/lstm_{}.model".format(epoch+1)
    torch.save(lstm, model_path)
    print("saved model: ", model_path)

epoch:1, step:10, loss:0.415027379989624
epoch:1, step:20, loss:0.429777055978775
epoch:1, step:30, loss:0.4007999897003174
epoch:1, step:40, loss:0.3955574929714203
epoch:1, step:50, loss:0.42692142724990845
epoch:1, step:60, loss:0.38942256569862366
epoch:1, step:70, loss:0.3879603445529938
epoch:1, step:80, loss:0.3830166757106781
epoch:1, step:90, loss:0.4048452377319336
epoch:1, step:100, loss:0.416843980550766
              precision    recall  f1-score   support

         0.0       0.78      0.70      0.74       155
         1.0       0.87      0.91      0.89       345

    accuracy                           0.85       500
   macro avg       0.83      0.81      0.81       500
weighted avg       0.84      0.85      0.84       500

准确率: 0.846
AUC: 0.9197755960729312
saved model:  ./model/lstm_1.model
epoch:2, step:10, loss:0.38801756501197815
epoch:2, step:20, loss:0.4014895558357239
epoch:2, step:30, loss:0.3345201015472412
epoch:2, step:40, loss:0.3716121315956116
epoch:2, step:

### 手动输入句子，判断情感倾向（1正/0负）

In [28]:
# torch.serialization.add_safe_globals([LSTM])    # 使用安全全局允许列表
net = torch.load("./model/lstm_5.model", weights_only=False)    # 训练过程中的巅峰时刻

In [29]:
from utils import processing

strs = ["我想说我会爱你多一点点", "日有所思梦感伤"]

data = []
for s in strs:
    vectors = []
    for w in processing(s).split(" "):
        if w in word2vec.wv.key_to_index:
            vectors.append(word2vec.wv[w])   # 将每个词替换为对应的词向量
    vectors = torch.Tensor(vectors)
    data.append(vectors)
x, _, lengths = collate_fn(list(zip(data, [-1] * len(strs))))
with torch.no_grad():
    x = x.to(device)
    outputs = lstm(x, lengths)          # 前向传播
    outputs = outputs.view(-1)          # 将输出展平
outputs

tensor([0.9805, 0.5076], device='cuda:0')